In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

In [2]:
ratings = pd.read_csv('ratings.csv')

In [3]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [4]:
ratings.drop(columns='timestamp',inplace=True)

In [5]:
ratings.columns = ['uid','iid','rating']

In [6]:
from surprise import KNNWithMeans, KNNBasic, Dataset, Reader, accuracy
from surprise.model_selection import GridSearchCV,train_test_split, cross_validate

In [7]:
# make dataset
rating_min,rating_max =  ratings.rating.min(),ratings.rating.max()
reader = Reader(rating_scale=(rating_min,rating_max) )
surprise_df = Dataset.load_from_df(ratings, reader)

In [17]:
# train_test_split .3
train_dataset, test_dataset = train_test_split(surprise_df, test_size=.3, )

#### KNN Means  User based

In [24]:
algo = KNNWithMeans(k=40, min_k=1, sim_options={'name': 'pearson_baseline', 'user_based': True}, verbose=True)
algo.fit(train_dataset)

In [30]:
y_pred = algo.test(test_dataset)
accuracy.rmse(y_pred)

RMSE: 0.4723


0.4722625212094781

#### KNN Means  Item based

In [35]:
algo = KNNWithMeans(k=50, min_k=1, sim_options={'name': 'pearson_baseline', 'user_based': False}, verbose=True)
algo.fit(train_dataset)
y_pred = algo.test(test_dataset)
accuracy.rmse(y_pred)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8885


0.8884994940489565

#### Проверим какой лучше на GridSearch CV

In [ ]:
params = {'k': [50,70,90,120,150], 'sim_options': {'name': ['cosine', 'msd','pearson','pearson_baseline'], 'user_based': [True,False]} }
grid = GridSearchCV(KNNWithMeans, params, measures = ['rmse'], cv = 5  )
grid.fit(surprise_df)
best_algo = grid.best_estimator['rmse']
best_algo.fit(train_dataset) 

In [31]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.8831946334859871}
{'rmse': {'k': 50, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}}}


In [32]:
y_pred = best_algo.test(test_dataset)
accuracy.rmse(y_pred)

RMSE: 0.8966


0.8966429416530666

In [33]:
result_df = pd.DataFrame.from_dict(grid.cv_results)

In [34]:
result_df.sort_values(by='rank_test_rmse',ascending = True).head(3)

,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_k,param_sim_options
7,0.891487,0.875628,0.881165,0.884136,0.883558,0.883195,0.005122,1,16.170823,0.113496,11.075816,0.118672,"{'k': 50, 'sim_options': {'name': 'pearson_bas...",50,"{'name': 'pearson_baseline', 'user_based': False}"
15,0.891609,0.875975,0.881143,0.884684,0.883885,0.883459,0.005089,2,16.108623,0.058445,12.008817,0.098812,"{'k': 70, 'sim_options': {'name': 'pearson_bas...",70,"{'name': 'pearson_baseline', 'user_based': False}"
23,0.891980,0.876001,0.881145,0.885185,0.884147,0.883692,0.005230,3,16.047823,0.023327,12.833818,0.159004,"{'k': 90, 'sim_options': {'name': 'pearson_bas...",90,"{'name': 'pearson_baseline', 'user_based': False}"


### Вывод по KNNwithMeans:  

    - Лучший результат: rootMSE = .88 , **Item based**  кол-во похожих фильмов  50 (к = 50) дистанция по Пирсону.
    - Второй результат: rootMSE = .88 , **Item based**  кол-во похожих фильмов  70 (к = 40) дистанция  по Пирсону.
    - Третий результат: rootMSE = .88 , **Item based**  кол-во похожих фильмов  90 (к = 90) дистанция по Пирсону.

### SVD (на основе скрытых рекомендаций)

In [10]:
from surprise import SVD,SVDpp

The prediction r^ui is set as:
        
        r^ui=μ+bu+bi+qTipu

If user u is unknown, then the bias bu and the factors pu are assumed to be zero. The same applies for item i with bi and qi.

To estimate all the unknown, we minimize the regularized squared error (через stochastic gradient descent) :

     ∑rui∈Rtrain(rui−r^ui)2+λ(b2i+b2u+||qi||2+||pu||2)
    

In [36]:
svd = SVD(n_factors=20,n_epochs=30).fit(train_dataset)
y_pred = svd.test(test_dataset)
accuracy.rmse(y_pred)

RMSE: 0.8758


0.8758495569954028

In [14]:
# SVDpp
svdpp = SVDpp(n_factors=20,n_epochs=30).fit(train_dataset)
y_pred = svdpp.test(test_dataset)
accuracy.rmse(y_pred)

RMSE: 0.8716


0.8715515699683009

**Метод SVD pp и SVD** (по сути один и тот же алгоритм работают на основе скрытых рекомендаций).  
Оба алгоритма показывают наилучшие результаты по **rootMSE = .87**

### KNN Basic  *(алгоритм который пытается оптимизировать функцию loss (root_MSE например) )*

**bsl_options**

method:

    - Using Stochastic Gradient Descent (SGD).
    - Using Alternating Least Squares (ALS).

    * ALS:
        - 'reg_i': The regularization parameter for items.
        - 'reg_u': The regularization parameter for users. 
        - 'n_epochs': The number of iteration of the ALS procedure.
    
    * SGD:
            - 'reg': The regularization parameter of the cost function
            - 'learning_rate': The learning rate of SGD
            - 'n_epochs': The number of iteration of the SGD procedure

In [57]:
# Проверим какой метод лучше на GridSearch CV

algo = KNNBasic()
param_grid_2 = {'bsl_options': {'method': ['als', 'sgd'],
                              'reg': [.1, .2],
                              'reg_i':[10,15],
                              'reg_u':[10,15],
                              'n_epochs':[5]},
              'sim_options': {'name': ['pearson_baseline', 'cosine'],
                              'user_based': [True,False]} }

In [58]:
grid_2 = GridSearchCV(KNNBasic , param_grid_2, measures = ['rmse'], cv = 3  )

In [ ]:
grid_2.fit(surprise_df)

In [60]:
print(grid_2.best_score)
print(grid_2.best_params)
best_algo = grid_2.best_estimator['rmse']
best_algo.fit(train_dataset)
y_pred = best_algo.test(test_dataset)
accuracy.rmse(y_pred)

{'rmse': 0.9157294444055979}
{'rmse': {'bsl_options': {'method': 'sgd', 'reg': 0.2, 'reg_i': 10, 'reg_u': 10, 'n_epochs': 5}, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}}}
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9145


0.9145079720300874

#### Выводы: 

Самый лучший результат **KNNBasic** rootMSE = 0.91 

> метод Стохасстический Градиентный Спуск (SGD) **Item Based** по Пирсону

### Проврим на датасете 1М:

In [37]:
ratings = pd.read_csv('../../ml-latest/ratings.csv')

In [38]:
ratings.drop(columns='timestamp',inplace=True)

In [39]:
ratings.columns = ['uid','iid','rating']

In [40]:
# make dataset
rating_min,rating_max =  ratings.rating.min(),ratings.rating.max()
reader = Reader(rating_scale=(rating_min,rating_max) )
surprise_df = Dataset.load_from_df(ratings, reader)

In [41]:
del ratings

Так как датасет большой то gridSearch делать будем только из двух "победителей" датасета на 100Кб

In [9]:
params = {'k': [40,50], 'sim_options': {'name': ['pearson_baseline'], 'user_based': [False]} }
grid_3 = GridSearchCV(KNNWithMeans, params, measures = ['rmse'], cv = 3  )

In [10]:
grid_3.fit(surprise_df)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...


MemoryError: Unable to allocate 9.15 GiB for an array with shape (49566, 49566) and data type int32

In [ ]:
print(grid_3.best_score)
print(grid_3.best_params)
best_algo = grid_3.best_estimator['rmse']

Обучим на трейн датасет и проверим на валидационном

In [ ]:
best_algo.fit(train_dataset)
y_pred = best_algo.test(test_dataset)
accuracy.rmse(y_pred)

In [ ]:
result_df = pd.DataFrame.from_dict(grid_3.cv_results)
result_df.sort_values(by='rank_test_rmse',ascending = True).head(3)

 **Не хватило памяти** просит 9.15Гб RAM под 1м датасет.  
Для 100кб датасета результаты в выводы смотрите ниже.

#### Попробуем прогнать SVD (на основе скрытых рекомендаций)
он считается быстрее всех и показывает наилучший результат: 

In [42]:
# train_test_split .15
train_dataset, test_dataset = train_test_split(surprise_df, test_size=.15 )
del surprise_df

In [43]:
svd = SVD(n_factors=20,n_epochs=30).fit(train_dataset)
y_pred = svd.test(test_dataset)
accuracy.rmse(y_pred)

RMSE: 0.7916


0.7915949123110518

### Выводы по всей работе: 

*Все результаты получены только на отложенной выборке. Самые лучшие параметры из GridSearch заново обучались на train и валидировались на test.*


На 100Кб датасете:

- Метод **SVD pp и SVD** rootMSE = .87  (по сути один и тот же алгоритм) Алгоритмы работают на основе скрытых рекомендаций, с заданным количеством скрытых факторов =20.
- Метод **KNNwithMeans** rootMSE = 0.88 (**Item based**  кол-во похожих фильмов  50 (к = 50) дистанция по Пирсону)
- Метод **KNNBasic** rootMSE = 0.91  (метод оптимизации функции потерь - Стохасстический Градиентный Спуск (SGD) **Item Based** по Пирсону)
    
Самый лучший результат на 100Кб датасете:  **SVD** rootMSE = **.87 **

На 1М датасете:

- запустить **KNNwithMeans** или **KNNBasic** не удалось - по нехватке памяти RAM ( просит 9.5 Гб)
- Метод **SVD** выдал rootMSE = .79  (на основе скрытых рекомендаций, с заданным количеством скрытых факторов =20).

И снова, самый лучший результат на 1М датасете:  **SVD** rootMSE = **.79 ** 